In [23]:
import pandas as pd
import numpy as np

# Load the dataset
ratings = pd.read_csv('./ml-20m/ratings.csv')
movies = pd.read_csv('./ml-20m/movies.csv')
links = pd.read_csv('./ml-20m/links.csv')
ratings.head()
ratings['userId'].nunique()


138493

In [24]:
# Filter to active users who has at least 50 ratings
user_rating_counts = ratings['userId'].value_counts()
active_users = user_rating_counts[user_rating_counts >= 50].index
print(f"Number of active users: {len(active_users)}")
len(ratings)


Number of active users: 85307


20000263

In [25]:
# Select a subset of 10k users
selected_users = active_users.to_series().sample(10000, random_state=7)
ratings_small = ratings[ratings['userId'].isin(selected_users)]
len(ratings_small)


2130396

In [26]:
# Clean movie and link
movies_small = movies[movies['movieId'].isin(ratings_small['movieId'])]
links_small = links[links['movieId'].isin(ratings_small['movieId'])]
summary = {
    "num_users": ratings_small['userId'].nunique(),
    "num_movies": ratings_small['movieId'].nunique(),
    "num_ratings": len(ratings_small)
}
summary


{'num_users': 10000, 'num_movies': 17614, 'num_ratings': 2130396}

In [27]:
# Export data
ratings_small.to_csv('./subset_ratings.csv', index=False)
movies_small.to_csv('./subset_movies.csv', index=False)
links_small.to_csv('./subset_links.csv', index=False)